In [ ]:
import pandas as pd
import numpy as np
import sys
df = pd.DataFrame(np.random.randn(10,3))
df = df.applymap(lambda x: round(x*10,0))

#np.random.randn(10,3)

df.iloc[3:5,0] = np.nan
df.iloc[4:6,1] = np.nan
df.iloc[5:8,2] = np.nan
df.info()
sys.getsizeof(df)

In [ ]:
from sklearn.decomposition import PCA


file_name = r'..\check_8_c\train.csv'
df = pd.read_csv(file_name, nrows=100, low_memory=False )
df_X = df.drop('target', axis=1)
number_columns = [
    col_name
    for col_name in df_X.columns
    if col_name.startswith('number')
]
df_X = df_X[number_columns]
df_X.fillna(-1, inplace=True)

pca = PCA()
pca
#pca.fit_transform(df_X)
#expl_var[ expl_var > max()]

In [ ]:
expl_var_ratio = pca.explained_variance_ratio_
i = 0
sum_ratio = 0
for x in expl_var_ratio:
    sum_ratio +=x
    i += 1
    if sum_ratio >.99:
        break
print(i)
np.sum(expl_var_ratio[:i])
len(pca.components_[0])
#expl_var = pca.explained_variance_
#max_expl_var = max(expl_var)
#expl_var[ expl_var > max_expl_var * 0.01]

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor()
a = {'a':1, 'b':2}

import os
d = pd.DataFrame([a])
header = not os.path.isfile('metrics.csv')
d.to_csv('metrics.csv', mode='a', index=False,header=header )


"""
from sklearn.linear_model import LogisticRegression
r = LogisticRegression()
r.predict()
"""

In [ ]:
import os
file_name = r'C:\ds\sdsj\check_8_c\train.csv'
file_size = os.path.getsize(file_name)

"""
df = pd.read_csv(file_name, nrows=100 )
df.to_csv('test_row_size.csv', header=False)
row_size = os.path.getsize('test_row_size.csv')
print('rows estimation', file_size/row_size)

usecols=['number_0','wefwefwewe']
usecols = df.columns & usecols
print(usecols)
"""

#df = pd.read_csv(file_name, usecols=usecols, nrows=1000 )
df = pd.read_csv(file_name, nrows=1000 )
#corr = df[ :df.shape[1] ].corr()
#df_corr = pd.DataFrame( corr )

#corr_cols = set()
#for i in range(df_corr.shape[0]):
#    row = df_corr.iloc[i]    
#    print(row[row>0.95])

    #cols = row.filter(lambda x: abs(x) > 0.95)
#        if abs(v) > hyper_params_corr_limit and i != j:
#            corr_cols[corr.columns[j]] = True            

print(corr_cols)

#df = pd.read_csv(r'C:\ds\sdsj\check_8_c\train.csv', nrows=10 )
#df.shape
#round(1234,-2)

In [ ]:
df.mean()

In [ ]:
#df.apply(lambda x: x.fillna(x.mean()), axis=0)
#df.fillna

df_X = df.copy()
df_X[2].value_counts().index[0]

#df_X[1].fillna( df_X[1].max(), inplace=True)
#df_X

In [ ]:
#train
argv = ['--train-csv', r'..\check_1_r\train.csv',
       '--model-dir', r'.',
       '--mode', 'regression']
print(' '.join(argv) )

In [ ]:
#predict
argv = ['--test-csv', r'..\check_1_r\test.csv',
        '--prediction-csv', r'..\check_1_r\prediction.csv', 
       '--model-dir', r'.']
print(' '.join(argv) )

In [ ]:
#score
argv = ['--test-target-csv', r'..\check_1_r\test-target.csv',
       '--prediction-csv', r'..\check_1_r\h2o_prediction.csv']
print(' '.join(argv) )

In [ ]:
%%time
import pandas as pd
import sys
import os
#from tpot import TPOTClassifier, TPOTRegressor
#tpotr = TPOTRegressor()
df = pd.read_csv(r'..\check_8_c\train.csv', nrows=None, low_memory=False)

In [ ]:
%%time
#if any(df.isnull()):    
#   df.fillna(-1, inplace=True)

print(df.isnull().values.any())
df.fillna(-1, inplace=True)
print(df.isnull().values.any())

In [ ]:
df.info(memory_usage='deep')

In [ ]:
%%time

def transform_ds(df):
    int_cols = []
    float_cols = []
    category_cols = []
    other_cols = []
    
    df_onehot = pd.DataFrame()

    for col_name in df.columns:
        
        n_uniq = df[col_name].nunique()
        if n_uniq == 1: # skip constant columns            
            continue
            
        col_type = df.dtypes[col_name]
        if col_type in ['int','int64']:
            int_cols.append(col_name)
        elif col_type in ['float', 'float64']:
            float_cols.append(col_name)
        elif col_type == 'object':
            uniq_values = df[col_name].unique()
            total = len( df[col_name] )
            if 2 < n_uniq <= 20:
                for uniq in uniq_values:
                    df_onehot['onehot_{}={}'.format(col_name,uniq)] = (df[col_name] == uniq).astype(int)
            elif n_uniq / total < 0.5: 
                category_cols.append(col_name)
            else:
                other_cols.append(col_name)
        else:
            other_cols.append(col_name)

    df_opt = df_onehot.apply(pd.to_numeric, downcast='integer')
    
    if len(int_cols) > 0:
        df_opt[int_cols] =  df[int_cols].apply(pd.to_numeric, downcast='integer')
    
    if len(float_cols) > 0:
        df_opt[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')
        
    if len(category_cols) > 0:
        df_opt[category_cols] = df[category_cols].astype('category')    
        
    if len(other_cols) > 0:
        df_opt[other_cols] = df[other_cols]

    return df_opt

df_opt = transform_ds(df)
df_opt.info(memory_usage='deep')

In [ ]:
df_opt.dtypes.value_counts()

In [ ]:
%%time

from sklearn.feature_selection import VarianceThreshold # mutual_info_regression
fs = VarianceThreshold(0.001)
df_nums = df_opt.select_dtypes(include=['float32','int8']) 
fs.fit(df_nums) # fit_transform(df)

supported = fs.get_support()
print('rows to remove:', len(supported[supported==False]))
#print( df_opt['number_6'].value_counts() )
for i in range(len(supported)):
    if not supported[i]:
        print(df_opt.columns[i])


In [ ]:
%%time
from sklearn.feature_selection import mutual_info_regression
df_X =df_nums.drop('target',axis=1)
df_y = df.target
mir = mutual_info_regression(df_X[:1000], df_y[:1000])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(mir)
#print(len(mir[mir>0.02]))

In [ ]:
print(df_X.columns[mir>np.mean(mir) + 4*np.std(mir)])

In [ ]:
constant_columns = [
        col_name
         for col_name in df_opt.columns
         if df_opt[col_name].nunique() == 1
        ]
constant_columns

In [ ]:
print(df_opt.isnull().values.any())

#df.fillna(-1, inplace=True)
#print(df.isnull().values.any())

In [ ]:
# %%time
import datetime


empty_date = datetime.datetime.strptime('0001-01-01', '%Y-%m-%d')


def parse_dt(x):
    if not isinstance(x, str):
        return empty_date
    elif len(x) == len('2010-01-01'):
        return datetime.datetime.strptime(x, '%Y-%m-%d')
    elif len(x) == len('2010-01-01 10:10:10'):
        return datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    else:
        return empty_date
    

def transform_datetime_features(df):
    datetime_columns = [
        col_name
        for col_name in df.columns
        if col_name.startswith('datetime')
    ]

    df_date = pd.DataFrame() #(dtypes=[['datetime','uint8','uint8','uint8','uint16']])
    for col_name in datetime_columns:
        #df_date[col_name],
        #df_date['number_weekday_{}'.format(col_name)],
        #df_date['number_month_{}'.format(col_name)],
        #df_date['number_day_{}'.format(col_name)],
        #df_date['number_hour_{}'.format(col_name)],
        #df_date['number_hour_of_week_{}'.format(col_name)],
        #df_date['number_minute_of_day_{}'.format(col_name)] = 
        df[col_name].apply(lambda x: parse_dt(x)[0] )
        
        #df_date[col_name] = a[0]
        #df_date['number_weekday_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.weekday())
        #df_date['number_month_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.month)
        #df_date['number_day_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.day)
        #f_date['number_hour_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.hour)
        #df_date['number_hour_of_week_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.hour + x.weekday() * 24)
        #df_date['number_minute_of_day_{}'.format(col_name)] = df_date[col_name].apply(lambda x: x.minute + x.hour * 60)        
        
    return transform_ds(df_date)

"""
apply(lambda x: parse_dt(x)) (FAST):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143525 entries, 0 to 143524
Columns: 427 entries, number_weekday_datetime_0 to datetime_90
dtypes: datetime64[ns](90), float32(337)
memory usage: 283.1 MB
Wall time: 6min 16s

pd.to_datetime (SLOW):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143525 entries, 0 to 143524
Columns: 630 entries, number_weekday_datetime_0 to datetime_90
dtypes: datetime64[ns](90), int16(174), int8(366)
memory usage: 196.3 MB
Wall time: 8min 34s
"""

#df_dates = transform_datetime_features(df_opt)
#df_dates.info(memory_usage='deep')
str(datetime.MINYEAR).rjust(4,'0')
str(20183).rjust(4,'0')

In [ ]:
#df_opt['datetime_3']
#d = pd.DataFrame(dtype=[['datetime','uint8','uint8','uint8','uint8','uint8','uint16']])
#type(df_opt['datetime_0'])
#pd.Series((None,None,None,None,None,None,None))
pd.__version__

In [ ]:
df_opt[df_dates.columns] = df_dates
df_opt.info(memory_usage='deep')

In [ ]:
opt_dates = transform_ds(pd.DataFrame())
#df_dates.info(memory_usage='deep')
#df_dates.head()

In [ ]:
df_opt.info(memory_usage='deep')
df_opt.head()

In [ ]:
%%time
df_int = df.select_dtypes(include=['int','int64'])
df_opt = df_int.apply(pd.to_numeric, downcast='unsigned')
print(df_opt.shape)
print(df_int)

In [ ]:
df_float = df.select_dtypes(include=['float'])
df_opt = pd.concat( [df_opt, df_float.apply(pd.to_numeric, downcast='float')], axis=1)
print(df_opt.shape)

In [ ]:
for col in df.columns:
    print(df.dtypes[col])

In [ ]:
def estimate_rows( file_name ):
    nrows = 200
    test_file_name = 'test_row_count.csv'

    file_size = os.path.getsize(file_name)
    df = pd.read_csv(file_name, nrows=nrows)
    df.to_csv(test_file_name, header=False)
    row_size = os.path.getsize(test_file_name) / nrows
    rows = file_size / row_size
    size = rows * sys.getsizeof(df) / nrows
    return { 'rows': int(rows), 'row_size': int(row_size), 'total_size': int(size) }

#estimate_rows(r'..\check_1_r\train.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.transform()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
reg = GradientBoostingRegressor()
cls = GradientBoostingClassifier()
print( reg.get_params().keys() )
print()
print( cls.get_params().keys() )

In [ ]:
from sklearn.model_selection import GridSearchCV
#?GridSearchCV
?GradientBoostingRegressor

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('test')
msg['Subject'] = 'subj'
msg['From'] = 'pomka@yandex.ru'
msg['To'] = 'pomka@yandex.ru'

HOST = "smtp.yandex.ru"
server = smtplib.SMTP(HOST)

username = 'pomka'
server.starttls()
server.login(username, input())
server.sendmail(msg['From'], msg['To'], msg.as_string())
server.quit()


In [ ]:
data_cols = rnd_cols.copy()
data_cols.append('zero')
#from sklearn.decomposition import PCA
#pca = PCA()
#pca.fit_transform(df)
#pca.explained_variance_ratio_
from sklearn.feature_selection import VarianceThreshold # mutual_info_regression
fs = VarianceThreshold(0.001)
a2 = fs.fit_transform(df[data_cols])
print(len(fs.variances_))
print(a2.shape)
#print( df.columns )
#GOOD:
#mir = mutual_info_regression(df[rnd_cols], df['multi']) !
#print(mir)

In [ ]:
%%time
import numpy as np
import pandas as pd

rows = 10**4
cols = 100
a = np.random.randn(rows, cols)
rnd_cols = list(map( str, range(1,cols+1) ))
#print(rnd_cols)
df = pd.DataFrame(a, columns=rnd_cols)

#.corr()
df['depend'] = (df['1']+df['2']) / (df['3'] * df['4'])
df['zero'] = np.zeros(rows)
#df.corr()

In [ ]:
%%time
from sklearn.feature_selection import mutual_info_regression
mir = mutual_info_regression(df[rnd_cols], df['depend'])
print(mir)
plt.plot(mir)

In [ ]:
print(df[rnd_cols].columns[mir>np.mean(mir)]) # + 1*np.std(mir)])

In [ ]:
print(mir[mir > 0.01])

In [ ]:
%%time
df_unique = df_opt.apply(lambda x: x.nunique())



In [ ]:
df_unique.sort_values('counts', inplace=True)

In [ ]:
df_unique

In [ ]:
df_unique = df_unique[df_unique > 2]
df_unique = df_unique[df_unique <= 20]
df_unique.sort_values(inplace=True)

In [ ]:
df_unique['number_597']

In [ ]:
import numpy as np
X=[0.6,0.4,0.58,0.1,-1130]
m=['a','b','c','d','e']
#sorted_scores = sorted(scores)[::-1]
#[x for x in X if x>np.mean(X)]
m[3:5]

In [ ]:
df = pd.read_csv(r'..\check_1_r\train.csv', nrows=None, low_memory=False)
constant_columns = [
        col_name
        for col_name in df.columns
        if df[col_name].nunique() == 1
    ]
constant_columns

In [ ]:
import time
from zipfile import ZipFile
folder = '003-no-ext-libs\\'
#%cd $folder
zip_file_name = folder + time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime()) + '_submission.zip'
    
files = [
    'metadata.json',
    'predict.py',
    'train.py',
    'utils.py']

with ZipFile(zip_file_name, mode='w') as submission:
    for file in files:
        submission.write(folder + file,arcname=file)
submission.close()    

In [ ]:
import pandas as pd
df =  pd.read_csv(r'..\check_3_r\train.csv')
if any(df.isnull()):
    df.fillna(-1, inplace=True)


In [ ]:
print(len(df.columns))
for n in range(100,1000,100):
    df_X = df[:n]
    df_unique = df_X.apply(lambda x: x.nunique())
    df_const = df_unique[df_unique == 1]

    print(n, len(df_X.columns) - len(df_const))
df.head()

In [ ]:
import sys
import traceback
try:
    raise Exception('ALL FEATURES DROPPED, STOPPING')
except BaseException as e:
    print(e)
    print(traceback.format_exc())


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
est = GradientBoostingClassifier()
type(est)


In [ ]:
import hashlib
columns = b'dwefwefwefwe'
hash_digest = hashlib.md5(columns).hexdigest()
print(hash_digest.upper())

In [ ]:
import numpy as np
!SET OMP_NUM_THREADS=4
size = 10000
a = np.random.random_sample((size, size))
b = np.random.random_sample((size, size))
n = np.dot(a,b)
#np.show_config()